Get the HOGS

In [1]:
from skimage.io import imshow, imread
from skimage.feature import hog
from skimage import exposure
import skimage.transform as transform
import skimage.io
import skimage.exposure
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [2]:
def read_data(folder_path):
    X = []
    y = []

    for class_label in range(43):
        class_folder = os.path.join(folder_path, str(class_label))
        
        for file_name in os.listdir(class_folder):
            if file_name.endswith(".ppm"):
                file_path = os.path.join(class_folder, file_name)
                
                # Code snippet for hogz
                img = skimage.io.imread(file_path, plugin='matplotlib')
                img_gray = skimage.color.rgb2gray(img)
                reshaped_img = transform.resize(img_gray, (40, 40))
                normalized_image = skimage.exposure.rescale_intensity(reshaped_img, in_range='image', out_range=(0, 1))
                hog_features, hog_image = hog(normalized_image, orientations=8, pixels_per_cell=(6, 6), cells_per_block=(2, 2), visualize=True)
                hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 5))
                hog_features_array = hog_features.reshape(-1)
                X.append(hog_features_array)
                y.append(class_label)

    return np.array(X), np.array(y)

folder_path_train = "/home/jovyan/DATA/dataset_folder_train_images/GTSRB/Final_Training/Images"

X,y = read_data(folder_path_train)

In [3]:
#random Forest
X_train_forest, X_test_forest, y_train_forest, y_test_forest = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled_forest = scaler.fit_transform(X_train_forest)
X_test_scaled_forest = scaler.transform(X_test_forest)

# Create and train the Random Forest model
random_forest = RandomForestClassifier()
random_forest.fit(X_train_scaled_forest, y_train_forest)

# Make predictions on the test set
y_pred_forest = random_forest.predict(X_test_scaled_forest)

# Evaluate the model
accuracy_forest = accuracy_score(y_test_forest, y_pred_forest)
print("Accuracy for Random Forest:", accuracy_forest)


Accuracy for Random Forest: 0.9450395307319561


In [4]:
#SVM
X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(X, y, test_size=0.2, random_state=42)

clf= make_pipeline(StandardScaler(),SVC(kernel="rbf", gamma= "auto"))
clf.fit(X_train_svm,y_train_svm)
y_pred_svm = clf.predict(X_test_svm)
accuracy_svm = accuracy_score(y_test_svm, y_pred_svm)
print("Accuracy for a SVM:", accuracy_svm)

Accuracy for a SVM: 0.9802346340219332
